In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install pyrsgis

In [3]:
######################### Import des libraries #######################

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset, DataLoader, sampler
from pyrsgis import raster
from pyrsgis.convert import changeDimension
from copy import deepcopy
import torch
import torch.nn as nn
import pandas as pd
import scipy.signal
import scipy.ndimage
import os
import numpy as np

Warning! matplotlib_scalebar library not found.


In [0]:
######################### Import des données (tout en 1 seule tile) #########################

#path = "/content/drive/My Drive/D4G/Seredou_20190307_v2.tif"
path = "/content/drive/My Drive/D4G/Seredou_20151203_v2.tif"
#path = "/content/drive/My Drive/D4G/Seredou_20170205_v2.tif"#encore des 4
#path = "/content/drive/My Drive/D4G/Diecke_20200215_crop.tif"
#path = "/content/drive/My Drive/D4G/Diecke_20170205_crop.tif"

In [0]:
######################### Import des données (1 seule tile) #########################

path = "/content/drive/My Drive/D4G/2_Seredou_2017/"

Test = "2_Image_Seredou_32bits_20170205/S2A_20170205_seredou_32bits.tif"
MSI = "2_MSI_Seredou_20170205/S2A_20170205_seredou_ZE_MSI_89.tif"
Truth = "GroundTruth_Seredou_20170205/GroundTruth_20170205_seredou_ZE_89.tif"

In [0]:
######################### Import des données (plusieurs tiles) #########################

os.chdir("/content/drive/My Drive/D4G/1_Seredou_2015/")
"""
#for testing
test_raw = os.listdir("Images_test")
MSI_raw = os.listdir("MSI_test")
truth_raw = os.listdir("GroundTruth_test")

Test = {int(k.split("_")[-1].split(".tif")[0].split("(")[0]):('Images_test/'+k) for k in test_raw}
MSI = {int(k.split("_")[-1].split(".tif")[0].split("(")[0]):('MSI_test/'+k) for k in MSI_raw}
Truth = {int(k.split("_")[-1].split(".tif")[0].split("(")[0]):('GroundTruth_test/'+k) for k in truth_raw}"""


#for prod
test_raw = os.listdir("1_Images_Seredou_2015_32bits")
MSI_raw = os.listdir("1_MSI_Seredou_2015")
truth_raw = os.listdir("1_GroundTruth_Seredou_2015_tiled")

Test = {int(k.split("_")[-1].split(".tif")[0]):('1_Images_Seredou_2015_32bits/'+k) for k in test_raw}
MSI = {int(k.split("_")[-1].split(".tif")[0]):('1_MSI_Seredou_2015/'+k) for k in MSI_raw}
Truth = {int(k.split("_")[-1].split(".tif")[0].split("(")[0]):('1_GroundTruth_Seredou_2015_tiled/'+k) for k in truth_raw}

In [0]:
######################### Fonctions auxiliaires #########################

def divide_test_in_squares(size, img):
    return [img[:,int(i % img.shape[0])*size:((int(i % img.shape[0])+1)*size), int(i // img.shape[0])*size:(int(i // img.shape[0])+1)*size] for i in range(int(img.shape[0] * img.shape[1] // size**2))]

def divide_image_in_squares(size, img):
    return [img[int(i % img.shape[0])*size:((int(i % img.shape[0])+1)*size), int(i // img.shape[0])*size:(int(i // img.shape[0])+1)*size] for i in range(int(img.shape[0] * img.shape[1] // size**2))]

def get_window(img, center, size):
    
    window = torch.tensor([0. for i in range(size*size)])
    if center[0] + size // 2 < img.shape[0]:
        i_begin = max(0,int(center[0]-size//2))
        i_end = i_begin + size
    else:
        i_end = img.shape[0]
        i_begin = i_end - size
    if center[1] + size // 2 < img.shape[1]:
        j_begin = max(0,int(center[1]-size//2))
        j_end = j_begin + size
    else:
        j_end = img.shape[1]
        j_begin = j_end - size

    for i in range(i_begin, i_end):
        for j in range(j_begin, j_end):
            window[i - i_begin + size*(j - j_begin)] = float(img[i,j])
    return window

In [0]:
######################### Base de données (1 image) #########################

class OurDataset(Dataset):
    """This dataset includes .... """
    
    def __init__(self, path, Test, MSI, Truth, size, percentage, transform=None):
        """
        Args:
            path: (str) path to the images directory.
        """
        #get data
        ds1, data_MSI = raster.read(path + MSI, bands='all')
        ds2, data_test = raster.read(path + Test, bands='all')
        ds3, data_truth = raster.read(path + Truth, bands='all')

        data_MSI[data_MSI < 0] = 0
        data_test[data_test < 0] = 0
        data_truth[data_truth < 0] = 0

        self.MSI = divide_image_in_squares(size, data_MSI)
        self.Test = divide_test_in_squares(size, data_test)
        self.Truth = divide_image_in_squares(size, data_truth)
  
        self.size = size
        self.mode = "train"
        self.id_train, self.id_test = train_test_split([i for i in range(len(self.Test))], test_size = percentage, random_state=42, shuffle=True)
        
    def __len__(self):
        if self.mode == "train":
            return len(self.id_train)
        else:
            return len(self.id_test)

    def __getitem__(self, id):
        """
        Args:
            idx: (int) the index of the subject/session whom data is loaded.
        Returns:
            sample: (dict) corresponding data described by the following keys:
                scan: 11 channels value
                mask: true value
        """
        if self.mode == "train":
            idx = self.id_train[id]
        else:
            idx = self.id_test[id]

        feature_data = torch.tensor([[0. for i in range(11)] for j in range(self.Test[idx].shape[1]*self.Test[idx].shape[2])])
        feature_truth = np.array([0 for j in range(self.Test[idx].shape[1]*self.Test[idx].shape[2])])

        #On importe les données
        for i in range(self.Test[idx].shape[1]):
            for j in range(self.Test[idx].shape[2]):
                feature_truth[i+self.Test[idx].shape[1]*j] = self.Truth[idx][i,j]
                feature_data[i+self.Test[idx].shape[2]*j,10] = float(self.MSI[idx][i,j])

                for k in range (10):
                    feature_data[i+self.Test[idx].shape[2]*j,k] = float(self.Test[idx][k,i,j])

        sample = {'data': feature_data, 'mask': feature_truth}

        return sample

    def train(self):
        """Put all the transforms of the dataset in training mode"""
        self.transform.train()

    def set_mode(self, mode):
        """Change mode of the database"""
        self.mode = mode

    def eval(self):
        """Put all the transforms of the dataset in evaluation mode"""
        self.transform.eval()

In [0]:
######################### Base de données (tout en 1 image) #########################

class OurDataset(Dataset):
    """This dataset includes .... """
    
    def __init__(self, path, size, percentage, transform=None):
        """
        Args:
            path: (str) path to the images directory.
        """
        #get data
        ds1, data = raster.read(path, bands='all')

        data[data < 0] = 0

        self.Test = divide_test_in_squares(size, data[:11,:,:])
        self.Truth = divide_image_in_squares(size, data[11,:,:])
        self.size = size
        self.mode = "train"
        self.id_train, self.id_test = train_test_split([i for i in range(len(self.Test))], test_size = percentage, random_state=42, shuffle=True)
        
    def __len__(self):
        if self.mode == "train":
            return len(self.id_train)
        else:
            return len(self.id_test)

    def __getitem__(self, id):
        """
        Args:
            idx: (int) the index of the subject/session whom data is loaded.
        Returns:
            sample: (dict) corresponding data described by the following keys:
                scan: 11 channels value
                mask: true value
        """
        if self.mode == "train":
            idx = self.id_train[id]
        else:
            idx = self.id_test[id]

        Test = self.Test[idx]

        feature_data = torch.tensor([[[0. for i in range(11)] for p in range(64)] for j in range(Test.shape[1] * Test.shape[2])])
        feature_truth = np.array([0 for j in range(Test.shape[1]*Test.shape[2])])

        #On importe les données
        for i in range(Test.shape[1]):
            for j in range(Test.shape[2]):
                feature_truth[i+self.Test[idx].shape[1]*j] = self.Truth[idx][i,j]
                
                for k in range (11):
                    test = get_window(Test[:,:,k], (i,j,k), 8)
                    for p in range(64):
                        feature_data[i+self.Test[idx].shape[1]*j,p,k] = test[p]
                        
        sample = {'data': feature_data, 'mask': feature_truth}

        return sample

    def train(self):
        """Put all the transforms of the dataset in training mode"""
        self.transform.train()

    def set_mode(self, mode):
        """Change mode of the database"""
        self.mode = mode

    def eval(self):
        """Put all the transforms of the dataset in evaluation mode"""
        self.transform.eval()

In [0]:
######################### Base de données (plusieurs images) #########################

class OurDataset(Dataset):
    """This dataset includes .... """
    
    def __init__(self, path, Test, MSI, Truth, percentage, transform=None):
        """
        Args:
            path: (str) path to the images directory.
        """
        self.MSI = {}
        self.Test = {}
        self.Truth = {}
        for key in MSI.keys():
            _, self.MSI[key] = raster.read(MSI[key], bands='all')
            _, self.Test[key] = raster.read(Test[key], bands='all')
            _, self.Truth[key] = raster.read(Truth[key], bands='all')

        self.mode = "train"
        self.id_train, self.id_test = train_test_split([k for k in list(self.Test.keys())], test_size = percentage, random_state=42, shuffle=True)
        
    def __len__(self):
        if self.mode == "train":
            return len(self.id_train)
        else:
            return len(self.id_test)

    def __getitem__(self, id):
        """
        Args:
            idx: (int) the index of the subject/session whom data is loaded.
        Returns:
            sample: (dict) corresponding data described by the following keys:
                scan: 11 channels value
                mask: true value
        """

        if self.mode == "train":
            idx = self.id_train[id] + 1
        else:
            idx = self.id_test[id] + 1

                #get data
        MSI = self.MSI[idx]
        Test = self.Test[idx]
        Truth = self.Truth[idx]
        
        MSI[MSI < 0] = 0
        Test[Test < 0] = 0
        Truth[Truth < 0] = 0
        
        feature_data = torch.tensor([[[0. for i in range(11)] for p in range(9)] for j in range(Test.shape[1] * Test.shape[2])])
        feature_truth = np.array([0 for j in range(Test.shape[1]*Test.shape[2])])

        #On importe les données
        for i in range(Test.shape[1]):
            for j in range(Test.shape[2]):
                feature_truth[i+self.Test[idx].shape[1]*j] = self.Truth[idx][i,j]
                
                msi = get_window(MSI, (i,j), 3)
                for p in range(9):
                    feature_data[i+self.Test[idx].shape[1]*j,p,10] = msi[p]
                                
                for k in range (10):
                    test = get_window(Test[:,:,k], (i,j,k), 3)
                    for p in range(9):
                        feature_data[i+self.Test[idx].shape[1]*j,p,k] = test[p]
                        
        sample = {'data': feature_data, 'mask': feature_truth}

        return sample

    def train(self):
        """Put all the transforms of the dataset in training mode"""
        self.transform.train()

    def set_mode(self, mode):
        """Change mode of the database"""
        self.mode = mode

    def eval(self):
        """Put all the transforms of the dataset in evaluation mode"""
        self.transform.eval()

In [0]:
def train(model, train_loader, criterion, optimizer, n_epochs, size):
    """
    Method used to train a CNN
    
    Args:
        model: (nn.Module) the neural network
        train_loader: (DataLoader) a DataLoader wrapping the dataset
        criterion: (nn.Module) a method to compute the loss of a mini-batch of images
        optimizer: (torch.optim) an optimization algorithm
        n_epochs: (int) number of epochs performed during training

    Returns:
        best_model: (nn.Module) the trained neural network
    """
    best_model = deepcopy(model)
    train_best_loss = np.inf

    batch_size = train_loader.batch_size
    n = 10

    n_batches = n//batch_size

    for epoch in range(n_epochs):
        model.train()
        total_loss = 0
        total_eq = 0
        total_nb = 0

        for i, data in enumerate(train_loader):
            images, mask = torch.reshape(data['data'], ((size**2)*batch_size,64,11)), data['mask']
            scans = torch.flatten(mask)
            outputs = torch.reshape(model(images), (scans.shape[0],4))
            loss = criterion(outputs, scans)
            loss.backward()
            soft_outputs = nn.functional.softmax(outputs)
            total_eq += int(torch.sum(torch.argmax(soft_outputs,1) == scans))
            total_loss += loss.item()
            total_nb += scans.shape[0]
            optimizer.step()
            optimizer.zero_grad()

        mean_loss = total_loss / len(train_loader.dataset)
        print('Epoch %i: loss = %f & accuracy = %f' % (epoch, mean_loss, total_eq/total_nb))

        if mean_loss < train_best_loss:
            best_model = deepcopy(model)
            train_best_loss = mean_loss
    
    return best_model

def test(model, data_loader, criterion, size):
    """
    Method used to test a CNN
    
    Args:
        model: (nn.Module) the neural network
        data_loader: (DataLoader) a DataLoader wrapping the dataset
        criterion: (nn.Module) a method to compute the loss of a mini-batch of images
    """
    model.eval()

    batch_size = data_loader.batch_size
    n = 10000

    n_batches = n//batch_size
    nb_true = 0
    nb_total = 0
    size_loss = 0
    total_loss = 0
    
    with torch.no_grad():
        for i, data in enumerate(data_loader):
            images, mask = torch.reshape(data['data'], ((size**2)*batch_size,64,11)), data['mask']
            scans = torch.flatten(mask)
            outputs = torch.reshape(model(images), (scans.shape[0],4))
            loss = criterion(outputs, scans)
            soft_outputs = nn.functional.softmax(outputs)
            total_loss += loss.item()
            size_loss += batch_size
            nb_true += int(torch.sum(torch.argmax(soft_outputs,1) == scans))
            nb_total += scans.shape[0]

    print("Final loss : {} & accuracy : {}".format(str(total_loss/ size_loss), str(nb_true/nb_total)))

In [0]:
########################## Modèle ############################

class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv1d(64, 64, 4)
        self.conv2 = nn.Conv1d(64, 64, 4)
        self.conv3 = nn.Conv1d(64, 64, 4)
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        self.lin1 = nn.Linear(128,64)
        self.lin2 = nn.Linear(64,32)
        self.lin3 = nn.Linear(32,4)
        self.lin4 = nn.Linear(16,16)
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.conv1(x)
        x = self.tanh(x)
        x = self.conv2(x)
        x = self.tanh(x)
        x = self.conv3(x)
        x = self.tanh(x)
        x = self.flatten(x)
        x = self.lin1(x)
        x = self.tanh(x)
        x = self.lin2(x)
        x = self.tanh(x)
        x = self.lin3(x)
        return x
        #return nn.functional.softmax(x)

In [0]:
########################## Database ############################

size = 20
database = OurDataset(path, size, 0.2)

In [0]:
########################## Prétraitement ############################

batch_size = 25

database.set_mode("train")
dataloader_train = DataLoader(database, batch_size=batch_size, drop_last=True)

model = Model()
#criterion = nn.CrossEntropyLoss()
criterion = nn.MultiMarginLoss(1)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
n_epochs = 5

In [0]:
#################### Entrainement du modèle ##########################

train(model, dataloader_train, criterion, optimizer, n_epochs, size)
torch.save(model.state_dict(), "model")

In [0]:
############################### Statistiques ################################

#Predict for test data 

database.set_mode("test")
dataloader_val = DataLoader(database, batch_size=batch_size, drop_last=True)

test(model, dataloader_val, criterion, size)

Final loss : 0.04109051704406738 & accuracy : 0.5236
